<a href="https://colab.research.google.com/github/CYBERIC-EA/Recommendation-System-w-Cosine-Sim./blob/main/ARS_w_Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter
import scipy as sp #pivot egineering
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob
import re

#ML model
from sklearn.metrics.pairwise import cosine_similarity


#default theme and settings
pd.options.display.max_columns

#warning hadle

warnings.filterwarnings("always")
warnings.filterwarnings("ignore")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
animePath = '/content/drive/MyDrive/Data Science/anime recommendation system/ANIME RECCOMENDATION SYSTEM V2/anime-dataset-2023.csv'
anime = pd.read_csv(animePath)
anime.head()



,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [4]:

print("dataset information :\n")
print(anime.info())

dataset information :

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non

For this particular recommendation system im going to select these relevant columns

In [5]:
RecoDf = anime[['Name','Synopsis']]
print("Missing Values in the Anime Dataset (%) : \n\n")
print(round(RecoDf.isnull().sum().sort_values(ascending=False)/len(RecoDf.index), 4 ) * 100)

print("\n Duplicate Values in the Anime Dataset (%) : \n\n")
print(RecoDf.duplicated().sum())

Missing Values in the Anime Dataset (%) : 


Name        0.0
Synopsis    0.0
dtype: float64

 Duplicate Values in the Anime Dataset (%) : 


0


In [6]:
def remove_punctuation_and_turn_lower(text):
    translator = str.maketrans('', '', string.punctuation)
    return (text.translate(translator)).lower()

#function used to treat the string list to the correct form(no punctuation or numbers or empty spaces,only stemmed strings in their lower case form)
def string_cleaning(string_list):
  strings_limpas = [remove_punctuation_and_turn_lower(string) for string in string_list]

  strings_limpas_no_numbers = [re.sub(r'\d', '', string) for string in strings_limpas]

  new_list = [item for item in strings_limpas_no_numbers if item]

  tokens_without_sw = [word for word in new_list if not word in stopwords.words('english')]

  ps = PorterStemmer()
  steemed_words = [ps.stem(w) for w in tokens_without_sw]


  return steemed_words

#treating the synopsis column:

RecoDf['Synopsis'] = RecoDf['Synopsis'].apply(lambda x: x.split())
RecoDf['Synopsis'] = RecoDf['Synopsis'].apply(string_cleaning)

print(RecoDf.head(3))

#applying bag of words technique
x_bow = RecoDf['Synopsis'].apply(lambda x: ' '.join(x))

                              Name  \
0                     Cowboy Bebop   
1  Cowboy Bebop: Tengoku no Tobira   
2                           Trigun   

                                            Synopsis  
0  [crime, timeless, year, human, expand, across,...  
1  [anoth, day, anoth, bounty—such, life, often, ...  
2  [vash, stamped, man, bounti, head, reason, he,...  


Recommendation System;

In [7]:
# Using CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=5000)

new_column = (vectorizer.fit_transform(x_bow)).toarray()

# Calculation of cosine similarity between the vectors
similarities = cosine_similarity(new_column)

In [8]:
def get_anime_recomendation(name):
    #getting the index of the anime on the dataframe
    index = (anime.index[anime['Name'] == name])[0]

    #separating the original list and the ordenated list
    normal_list = similarities[index]
    ordenated_list = sorted(similarities[index],reverse=True)

    ranking_list = []
    for i in range(len(ordenated_list)):
        index_new = np.where(normal_list == ordenated_list[i])[0]
        tuple_anime = (int(index_new[0]),ordenated_list[i])
        ranking_list.append(tuple_anime)

    #getting top 5 anime recommendations:
    print('Top 5 recommendations based on the selected anime:\n')
    for tuple_values in ranking_list[1:6]:
        print(RecoDf.iloc[tuple_values[0]]['Name'])

In [9]:
get_anime_recomendation('Baki')

Top 5 recommendations based on the selected anime:

Baki: Dai Raitaisai-hen
Baki: Most Evil Death Row Convicts Special Anime
Grappler Baki: Saidai Tournament-hen
Beyblade Burst God
Ghost Messenger
